In [1]:
#!/usr/bin/env python
# coding: utf-8

import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

#conf = SparkConf().setAppName('Insight_Ingest_NLP_LogistcRgreesion1')
#.setMaster('yarn')
#sc = pyspark.SparkContext(conf=conf).getOrCreate()
#spark = SparkSession(sc)
spark = SparkSession.builder.appName('small_nlp_logistic').getOrCreate()

from pyspark.ml.feature import HashingTF, IDF, Tokenizer, CountVectorizer
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator

df_review = spark.read.json("s3://cf-templates-efn3brqcqavf-us-east-2/yelp/yelp_academic_dataset_review.json")
#df_review = df_review.limit(500)

import pyspark.sql.functions as f
df_review = df_review.filter("cool >=3 or useful >=3 or funny >=3") 
df_review = df_review.select("stars","text")
df_review = df_review.repartition(100)

from pyspark.sql import functions as F
df_review = df_review.withColumn("target", F.when( df_review.stars <=2,1 ).otherwise(0))


(train_set, test_set) = df_review.randomSplit([0.7, 0.3], seed = 1002)

from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline

tokenizer = Tokenizer(inputCol="text", outputCol="words")
cv = CountVectorizer(vocabSize=2**16, inputCol="words", outputCol='cv')
idf = IDF(inputCol='cv', outputCol="features", minDocFreq=5) #minDocFreq: remove sparse terms
label_stringIdx = StringIndexer(inputCol = "target", outputCol = "label")
#lr = LogisticRegression(maxIter=10)
pipeline = Pipeline(stages=[tokenizer, cv, idf, label_stringIdx])

pipelineFit = pipeline.fit(train_set)
train_df = pipelineFit.transform(train_set)
val_df = pipelineFit.transform(test_set)

lr = LogisticRegression(maxIter=100)
lrModel = lr.fit(train_df)
predictions = lrModel.transform(val_df)
#predictions = predictions.select('target','label', 'rawPrediction', 'probability', 'prediction')

from pyspark.sql.functions import col
prediction_1 = predictions.select('prediction','label','target',col("probability").cast("string"))
prediction_1.repartition(1).write.mode('overwrite').format('com.databricks.spark.csv').save("s3://cf-templates-efn3brqcqavf-us-east-2/yelp/output/logisticReg_pre.csv")

from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction")
from pyspark.sql.types import FloatType
eva_pre = evaluator.evaluate(predictions)
df_eva = spark.createDataFrame([eva_pre], FloatType())
df_eva.write.mode('overwrite').option("header", "true").format('com.databricks.spark.csv').save("s3://cf-templates-efn3brqcqavf-us-east-2/yelp/output/auc.csv")

evaluator.getMetricName()
accuracy = predictions.filter(predictions.label == predictions.prediction).count() / float(test_set.count())
df_acc = spark.createDataFrame([accuracy], FloatType())
df_acc.write.mode('overwrite').option("header", "true").format('com.databricks.spark.csv').save("s3://cf-templates-efn3brqcqavf-us-east-2/yelp/output/acc.csv")



VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
22,application_1601352654767_0072,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
from pyspark.sql.types import StringType, FloatType
list_coef = [float(round(x,4)) for x in lrModel.coefficients]
list_word = pipelineFit.stages[1].vocabulary
df_word_list = spark.createDataFrame(list_word,schema = StringType())
df_word_list.write.mode('overwrite').option("header", "true").format('com.databricks.spark.csv').save("s3://cf-templates-efn3brqcqavf-us-east-2/yelp/output/word_list.csv")
sc.parallelize(list_coef).coalesce(1).saveAsTextFile("s3://cf-templates-efn3brqcqavf-us-east-2/yelp/output/coef_list.txt")

#coe = lrModel.coefficients
#list_1 = []
#for i in range(len(lrModel.coefficients)):
#    list_temp = (str(pipelineFit.stages[1].vocabulary[i]),float(round(coe[i],5)))
#    list_1.append(list_temp)
#from pyspark.sql.types import FloatType, StringType
#df_coef = sqlContext.createDataFrame(list_1,["word","coef"]).filter("coef>=0.1").orderBy("coef", ascending = False)
#df_coef.write.mode('overwrite').format('com.databricks.spark.csv').save("s3://cf-templates-efn3brqcqavf-us-east-2/yelp/output/logisticReg_coef.csv")

lrModel.save("s3://cf-templates-efn3brqcqavf-us-east-2/yelp/output/model/lrmodel_neo")
print(accuracy)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.5652173913043478

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+
|     value|
+----------+
|0.76645875|
+----------+

In [76]:
#spark.createDataFrame(list(pipelineFit.stages[1].vocabulary), schema = ("words"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'NoneType' object has no attribute 'show'
Traceback (most recent call last):
AttributeError: 'NoneType' object has no attribute 'show'



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'list' object has no attribute 'toDF'
Traceback (most recent call last):
AttributeError: 'list' object has no attribute 'toDF'



In [74]:
from pyspark.sql.types import StringType, FloatType
list_coef = [float(round(x,4)) for x in lrModel.coefficients]
list_word = pipelineFit.stages[1].vocabulary
df_word_list = spark.createDataFrame(list_word,schema = StringType())
df_word_list.write.mode('overwrite').option("header", "true").format('com.databricks.spark.csv').save("s3://cf-templates-efn3brqcqavf-us-east-2/yelp/output/word_list.csv")
sc.parallelize(list_coef).coalesce(1).saveAsTextFile("s3://cf-templates-efn3brqcqavf-us-east-2/yelp/output/coef_list.txt")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [56]:
spark

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[0.0211, 0.0189, 0.0016, 0.0157, 0.0046, 0.0209, 0.0072, 0.005, -0.0003, 0.0135, 0.0064, 0.003, 0.0082, 0.0005, 0.0087, 0.0141, 0.0082, 0.0028, 0.0119, 0.0202, 0.0007, 0.0163, 0.0132, 0.0255, 0.0182, 0.0076, -0.0014, 0.0072, -0.0094, 0.0158, 0.0069, 0.0095, 0.0091, 0.0089, 0.0131, 0.0193, 0.0037, 0.006, 0.0108, 0.0146, 0.0075, 0.0159, -0.0155, 0.0027, 0.0104, 0.0097, 0.0112, 0.0046, 0.0066, 0.0108, 0.0137, 0.0226, -0.0126, 0.0025, 0.007, 0.0114, 0.018, -0.0025, 0.0094, 0.0168, 0.0081, -0.0081, 0.008, 0.0043, 0.0001, -0.0039, -0.003, 0.0122, 0.0199, 0.0074, 0.0017, 0.0173, -0.0109, 0.0077, 0.0018, 0.0172, 0.0082, 0.0135, -0.004, 0.0039, -0.0038, 0.0103, 0.0041, 0.0094, -0.0076, 0.0187, 0.0101, -0.0016, 0.0128, 0.0099, 0.0006, -0.0019, -0.0117, 0.0019, 0.0015, -0.011, 0.0117, 0.0055, 0.002, 0.0196, -0.0103, 0.0073, 0.0302, -0.009, -0.0011, 0.0052, -0.0167, 0.0096, 0.0089, 0.0217, 0.0116, 0.0008, 0.0116, 0.0101, 0.0055, -0.0032, 0.0082, -0.0139, -0.0074, 0.0164, 0.0054, 0.0025, 0.008, -0.

In [2]:
predictions.columns
#prediction_s = predictions.select("")#

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['stars', 'text', 'target', 'words', 'cv', 'features', 'label', 'rawPrediction', 'probability', 'prediction']

In [18]:
pipelineFit.stages[1].vocabulary

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'dinner,'

In [11]:
len(lrModel.coefficients)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

5310

In [17]:
predictions.select("cv","words").show(1, False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|cv                                                                                                        

In [24]:
spark.createDataFrame(zip((lrModel.coefficients).map(lambda x: float(x)),pipelineFit.stages[1].vocabulary),schema = ['importance','word']).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'SparseVector' object has no attribute 'map'
Traceback (most recent call last):
AttributeError: 'SparseVector' object has no attribute 'map'



In [3]:
list_temp = []
for i in range(len(lrModel.coefficients)):
    list_temp.append([float(lrModel.coefficients[i]),pipelineFit.stages[1].vocabulary[i]])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
Invalid status code '400' from https://172.31.33.190:18888/sessions/20/statements/3 with error payload: {"msg":"requirement failed: Session isn't active."}


In [4]:
spark.createDataFrame(list_temp,schema =['importance','word'])).show()

VBox()

An error was encountered:
Session 20 unexpectedly reached final status 'killed'. See logs:
stdout: 

stderr: 
20/10/06 20:27:23 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_13_6 !
20/10/06 20:27:23 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_13_12 !
20/10/06 20:27:23 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_13_72 !
20/10/06 20:27:23 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_13_71 !
20/10/06 20:27:23 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_13_30 !
20/10/06 20:27:23 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_13_78 !
20/10/06 20:27:23 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_13_56 !
20/10/06 20:27:23 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_13_43 !
20/10/06 20:27:23 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_13_84 !
20/10/06 20:27:23 WARN BlockManagerMasterEndp

In [14]:
pipelineFit.stages[1].vocabulary[0]
lrModel.coefficients

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparseVector(5310, {0: 0.0246, 1: -1.5396, 2: -0.0438, 3: -0.9626, 4: -0.2484, 5: 0.4826, 6: -0.1577, 7: 0.3658, 8: -1.2196, 9: -0.5708, 10: 0.1626, 11: -0.7547, 12: -0.799, 13: -0.0315, 14: -0.3086, 15: -0.3272, 16: 0.096, 17: 0.4093, 18: 0.0909, 19: -0.2143, 20: 0.8407, 21: -0.0627, 22: 0.4811, 23: -0.3483, 24: -0.3517, 25: 0.9927, 26: -0.2972, 27: -0.0869, 28: -1.5687, 29: -0.5912, 30: -0.1058, 31: -0.6843, 32: 0.2838, 33: -0.2208, 34: -0.0784, 35: 0.1684, 36: 0.2385, 37: 0.1001, 38: 0.2742, 39: 0.0905, 40: -0.6473, 41: -1.1036, 42: 0.2238, 43: -2.183, 44: -0.1966, 45: -0.0049, 46: 0.3358, 47: 0.7602, 48: 0.0148, 49: -0.509, 50: 0.3689, 51: 0.0768, 52: -0.0437, 53: -0.6471, 54: 0.6204, 55: 0.3399, 56: -0.3371, 57: -0.4556, 58: 0.2843, 59: 0.2566, 60: -0.7862, 61: 1.2467, 62: 0.3664, 63: -2.2345, 64: 1.5944, 65: -0.2962, 66: 0.3272, 67: -2.0018, 68: -0.4702, 69: -0.6317, 70: 0.2369, 71: 0.4949, 72: -0.2543, 73: -0.2061, 74: -0.2388, 75: -0.1583, 76: 0.4106, 77: 1.1395, 78: 0.3141, 79